In [ ]:
import joblib
import time
import concurrent.futures
import math
import torch
from collections import Counter
import time
from ML_Functions import *

In [ ]:
X_all = torch.load('Data/A4_virdim_dim_X.pt')
Y_all = torch.load('Data/A4_virdim_dim_Y.pt')
print(X_all.shape)
print(Y_all.shape)

# SVM

In [ ]:
weight_list = []     
bias_list = []
acc_list = []
for i in range(100):
    print(i)
    net,pre,acc = SVM_train(X_all,Y_all,ls_lambda=0.001,learning_rate=0.01,N_epoch=10,batch_size=1024,opt_adam=True,print_mid_loss=True)
    weight_list.append(net.linear1.weight.data)
    bias_list.append(net.linear1.bias.data)
    acc_list.append(acc)
    print('------------------------------------------')

In [ ]:
weight_all = torch.cat(weight_list,dim=0)
weight_avg = weight_all.mean(0)
acc_avg = sum(acc_list) / len(acc_list)
print('Coefficient = ' + ', '.join(['{:.2f}'.format(i) for i in torch.round(weight_avg * 100) / 100]))
print(acc_avg)

# Neural Network

In [ ]:
Y_all = Y_all*0.5+0.5   # form [-1,1] to [0,1] for Classification
X_tr,X_te,Y_tr,Y_te = Dataset_spit(X_all,Y_all,ratio=0.8)
print(X_tr.shape)
print(X_te.shape)
print(Y_tr.shape)
print(Y_te.shape)

In [ ]:
# Single-layer ReLU network with 10 neurons

sen_list = []
acc_list = []
acc_test_list = []
for i in range(100):
    print(i)
    net,acc = CLS_N_train_GPU(X_tr,Y_tr,n_class=2,n_layer=1,c_mid=10,learning_rate=0.01,weight_decay=0.001,N_epoch=50,batch_size=1024,opt_adam=True,print_mid_loss=True)
    acc_test = CLS_test_GPU(X_te,Y_te,net,batch_size=1024)
    sen_vec = CLS_grad_GPU(X_te,Y_te,net,batch_size=1024)
    sen_list.append(sen_vec.unsqueeze(dim=0).cpu().data)
    acc_list.append(acc)
    acc_test_list.append(acc_test)
    print('------------------------------------------')
sen_all = torch.cat(sen_list,dim=0)
sen_avg = sen_all.mean(0)
print('Avg sen:')
print('Coefficient = ' + ', '.join(['{:.2f}'.format(i) for i in torch.round(sen_avg.squeeze()*10000) / 10000]))
print(sum(acc_list) / len(acc_list))
print(sum(acc_test_list) / len(acc_list))

In [ ]:
# Three-layer ReLU network with 20 neurons

sen_list = []
acc_list = []
acc_test_list = []
for i in range(100):
    print(i)
    net,acc = CLS_N_train_GPU(X_tr,Y_tr,n_class=2,n_layer=3,c_mid=20,learning_rate=0.01,weight_decay=0.001,N_epoch=100,batch_size=1024,opt_adam=True,print_mid_loss=True)
    acc_test = CLS_test_GPU(X_te,Y_te,net,batch_size=1024)
    sen_vec = CLS_grad_GPU(X_te,Y_te,net,batch_size=1024)
    sen_list.append(sen_vec.unsqueeze(dim=0).cpu().data)
    acc_list.append(acc)
    acc_test_list.append(acc_test)
    print('------------------------------------------')
sen_all = torch.cat(sen_list,dim=0)
sen_avg = sen_all.mean(0)
print('Avg sen:')
print('Coefficient = ' + ', '.join(['{:.2f}'.format(i) for i in torch.round(sen_avg.squeeze()*10000) / 10000]))
print(sum(acc_list) / len(acc_list))
print(sum(acc_test_list) / len(acc_list))